In [1]:
pip install keras-tuner

In [13]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [14]:
print(tf.__version__)

2.19.0


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import os
# print(os.getcwd())
os.chdir('/content/drive/MyDrive/Colab_Notebooks')

In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = '/content/drive/MyDrive/Colab_Notebooks/chest_xray/train'
test_dir = '/content/drive/MyDrive/Colab_Notebooks/chest_xray/test'

train_datagen = ImageDataGenerator(
    rescale=1./255,          # Normalizes pixel values (0-255 → 0-1)
    rotation_range=20,       # Randomly rotate image within ±20 degrees
    width_shift_range=0.2,   # Random horizontal shift (20% of width)
    height_shift_range=0.2,  # Random vertical shift (20% of height)
    shear_range=0.2,         # Random shear transform (like slanting)
    zoom_range=0.2,          # Random zoom in/out (up to 20%)
    horizontal_flip=True,    # Flip images horizontally randomly
    fill_mode="nearest"      # Fill empty pixels by nearest values after transform
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # ✅ only H, W
    batch_size=8,
    class_mode='binary'  # ✅ if model output is 1 neuron sigmoid
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),  # ✅ must match train target size
    batch_size=8,
    class_mode='binary',
    shuffle=False
)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [19]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab_Notebooks/chest_xray/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

Found 16 images belonging to 2 classes.


In [20]:
def build_model(hp):
    model = keras.Sequential([
        keras.layers.Conv2D(
            filters=hp.Int('convo_1_filter', min_value=32, max_value=64, step=16),
            kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),
            activation='relu',
            input_shape=(224, 224, 3)   # Only here!
        ),
        keras.layers.MaxPooling2D(pool_size=(2, 2)), # Added MaxPooling2D
        keras.layers.Dropout(0.25),
        
        keras.layers.Conv2D(
            filters=hp.Int('convo_2_filter', min_value=16, max_value=32, step=16),
            kernel_size=hp.Choice('conv_2_kernel', values=[3, 5]),
            activation='relu'
        ),
        keras.layers.MaxPooling2D(pool_size=(2, 2)), # Added MaxPooling2D
        keras.layers.Dropout(0.25), #Added dropout to remove over fitting.
        
        keras.layers.Flatten(),
        keras.layers.Dense(
            units=hp.Int('dense_1_units', min_value=16, max_value=64, step=16), # Reduced max_value
            activation='relu'
        ),
        keras.layers.Dropout(0.25),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', values=[1e-3, 1e-4]) # Reduced learning rate range
        ),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

In [21]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [22]:
!rm -rf output/

In [23]:
tuner_search = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,       # ✅ Recommended (runs each config once)
    directory='output',
    project_name="Pneumonia_Prediction"
)

In [24]:
tuner_search.search(
    train_generator,
    epochs=5,                     # ✅ Increase for better tuning
    validation_data=validation_generator
)

Trial 5 Complete [00h 10m 09s]
val_accuracy: 0.875

Best val_accuracy So Far: 0.875
Total elapsed time: 00h 51m 28s


In [25]:
# Get the best model from tuner
from keras import backend as K
K.clear_session()
model = tuner_search.get_best_models(num_models=1)[0]

# View model structure
model.summary()

# Continue training best model
history = model.fit(
    train_generator,
    epochs=3,
    validation_data=validation_generator,
    verbose=1
)

# Save model
model.save("best_pneumonia_model.h5")
print("✅ Model Saved Successfully")


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 220, 220, 64)   │         4,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 110, 110, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 110, 110, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 106, 106, 16)   │        25,616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 53, 53, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 53, 53, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 44944)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │       719,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 749,617 (2.86 MB)

 Trainable params: 749,617 (2.86 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/3
652/652 ━━━━━━━━━━━━━━━━━━━━ 130s 192ms/step - accuracy: 0.8329 - loss: 0.3771 - val_accuracy: 0.6875 - val_loss: 0.5910
Epoch 2/3
652/652 ━━━━━━━━━━━━━━━━━━━━ 124s 190ms/step - accuracy: 0.8226 - loss: 0.3641 - val_accuracy: 0.8750 - val_loss: 0.5441
Epoch 3/3
652/652 ━━━━━━━━━━━━━━━━━━━━ 124s 190ms/step - accuracy: 0.8405 - loss: 0.3329 - val_accuracy: 0.8125 - val_loss: 0.5342


✅ Model Saved Successfully


In [26]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


78/78 ━━━━━━━━━━━━━━━━━━━━ 8s 87ms/step - accuracy: 0.5420 - loss: 0.9241
Test Accuracy: 0.7003
Test Loss: 0.6115
